In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import imageio

In [2]:
stress = pd.read_csv('data_stress/ASI_AnnualSummary_Season1_data.csv')

In [3]:
stress

,CROP_MASK,COUNTRY,ADM1_CODE,PROVINCE,YEAR,DATA,UNIT
0,Cropland,Armenia,NaN,ALL,2024,1.35,VHI % below 35
1,Cropland,Armenia,NaN,ALL,2023,5.01,VHI % below 35
2,Cropland,Armenia,NaN,ALL,2022,11.05,VHI % below 35
3,Cropland,Armenia,NaN,ALL,2021,22.66,VHI % below 35
4,Cropland,Armenia,NaN,ALL,2020,6.23,VHI % below 35
...,...,...,...,...,...,...,...
487,Cropland,Armenia,460.0,Shirak,1984,0.00,VHI % below 35
488,Cropland,Armenia,461.0,Syunik,1984,0.00,VHI % below 35
489,Cropland,Armenia,462.0,Tavush,1984,0.00,VHI % below 35
490,Cropland,Armenia,463.0,Vayots Dzor,1984,0.00,VHI % below 35


In [4]:
# Pivot so provinces are rows and years are columns, values from the DATA column
pivot_df = stress.pivot_table(index='PROVINCE', columns='YEAR', values='DATA', aggfunc='mean')

# sort provinces and years for nicer display
pivot_df = pivot_df.sort_index().sort_index(axis=1)

# show the result
pivot_df

YEAR,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
PROVINCE,,,,,,,,,,,,,,,,,,,,,
ALL,0.03,31.64,4.10,30.63,0.23,78.86,54.18,13.21,6.78,0.0,...,5.43,0.62,2.51,2.05,6.42,6.23,22.66,11.05,5.01,1.35
Aragatsotn,0.21,44.43,9.40,32.54,0.00,100.00,43.67,9.83,1.08,0.0,...,3.13,0.21,3.67,3.24,1.29,1.18,21.83,21.94,7.02,4.54
Ararat,0.00,43.51,5.45,18.97,0.00,98.58,43.62,44.60,7.63,0.0,...,1.09,0.65,1.52,7.63,7.63,4.03,17.12,30.09,12.86,2.18
Armavir,0.18,79.11,11.17,3.99,0.00,96.63,19.61,23.79,2.99,0.0,...,13.44,6.08,17.43,7.08,14.44,4.08,33.51,51.04,17.25,6.44
Gergharkunik,0.00,43.51,2.40,84.52,0.00,46.22,60.47,2.86,1.48,0.0,...,11.69,0.00,0.35,0.15,10.01,3.26,44.38,6.84,0.25,0.00
Kotayk,0.00,49.62,6.78,54.75,0.00,99.69,80.46,28.35,37.78,0.0,...,1.96,0.00,1.13,1.05,1.58,0.07,14.55,7.76,3.39,0.83
Lori,0.00,2.59,2.82,18.04,0.00,61.61,53.74,0.44,9.35,0.0,...,0.22,0.00,0.00,0.00,0.59,0.14,0.37,0.00,0.00,0.00
Shirak,0.00,9.33,1.64,33.97,0.00,99.70,48.41,0.00,0.00,0.0,...,0.35,0.00,0.11,0.00,0.11,0.29,14.14,0.99,0.00,0.00
Syunik,0.00,12.64,0.00,6.67,1.09,73.13,61.16,15.60,3.29,0.0,...,10.44,0.00,1.67,1.57,12.02,19.41,26.86,1.81,7.77,0.09


In [5]:
# import armenia shapefile
gdf = gpd.read_file('armenia_shapefile/am.json')

In [6]:
# map index names in pivot_df so they match the shapefile 'name' values (adjust mapping as needed)
mapping = {
    'Gergharkunik': 'Gegharkunik',
    'Yerevan': 'Erevan',
    # add more mappings here if you find other mismatches
}

pivot_df = pivot_df.rename(index=mapping)

# update reset version used for merging
pivot_reset = pivot_df.reset_index()

# quick checks
print("Current pivot_df index:", list(pivot_df.index))
print("Names in pivot not in gdf:", sorted(set(pivot_df.index) - set(gdf['name'])))

Current pivot_df index: ['ALL', 'Aragatsotn', 'Ararat', 'Armavir', 'Gegharkunik', 'Kotayk', 'Lori', 'Shirak', 'Syunik', 'Tavush', 'Vayots Dzor', 'Erevan']
Names in pivot not in gdf: ['ALL']


In [7]:
# save the pivoted DataFrame to CSV
pivot_df.to_csv('drought_data_output/agri_stress_pivoted.csv')

# Visual Representation
This first cell commented out is just printing every single image

In [8]:
# Load Armenia admin boundaries and plot pivoted agricultural-stress by year


# Preconditions
if 'pivot_df' not in globals():
    raise RuntimeError('pivot_df not found - run the pivot cell first')

 # Read the Armenia GeoJSON/shapefile (uses am.json if available)
gpath_json = 'armenia_shapefile/am.json'
gpath_shp = 'armenia_shapefile/am.shp'
if os.path.exists(gpath_json):
    gdf = gpd.read_file(gpath_json)
elif os.path.exists(gpath_shp):
    gdf = gpd.read_file(gpath_shp)
else:
    raise RuntimeError('No armenia shapefile/json found in armenia_shapefile/')

# Ensure CRS set (likely EPSG:4326)
if gdf.crs is None:
    gdf = gdf.set_crs(epsg=4326)

# Prepare pivot table for merging: reset index so province name is a column
pivot_reset = pivot_df.reset_index()

# detect the index name (usually 'PROVINCE')
idx_name = pivot_df.index.name if pivot_df.index.name is not None else 'PROVINCE'
if idx_name not in pivot_reset.columns:
    pivot_reset = pivot_reset.rename(columns={pivot_reset.columns[0]: idx_name})

 # Choose a colormap and fixed range 0..100
cmap = plt.get_cmap('viridis')
vmin, vmax = 0, 100

# # Loop year-by-year (columns of pivot_df) and plot
# years = list(pivot_df.columns)
# for year in years:
#     df_year = pivot_reset[[idx_name, year]].rename(columns={year: 'value'})
#     # merge on likely name column in the shapefile (commonly 'name')
#     if 'name' in gdf.columns:
#         merged = gdf.merge(df_year, left_on='name', right_on=idx_name, how='left')
#     else:
#         # fallback: try to merge on the first non-geometry column
#         left_key = [c for c in gdf.columns if c.lower() != 'geometry'][0]
#         merged = gdf.merge(df_year, left_on=left_key, right_on=idx_name, how='left')

#     fig, ax = plt.subplots(figsize=(8, 8))
#     merged.plot(column='value', ax=ax, cmap=cmap, vmin=vmin, vmax=vmax,
#                 edgecolor='black', linewidth=0.5, legend=True,
#                 missing_kwds={'color': 'lightgrey', 'label': 'no data'})
#     ax.set_title(f'Annual agricultural stress — {year}')
#     ax.axis('off')
#     plt.show()

# # If labels are wanted, you can add representative_point labels similar to the previous notebook cell.

In [9]:
gdf

,source,id,name,geometry
0,https://simplemaps.com,AMTV,Tavush,"POLYGON ((45.37359 40.6513, 45.37333 40.65137,..."
1,https://simplemaps.com,AMLO,Lori,"POLYGON ((44.85382 41.22351, 44.85393 41.2235,..."
2,https://simplemaps.com,AMSH,Shirak,"POLYGON ((43.97776 41.16454, 43.97755 41.16315..."
3,https://simplemaps.com,AMGR,Gegharkunik,"MULTIPOLYGON (((44.79657 40.65008, 44.91543 40..."
4,https://simplemaps.com,AMVD,Vayots Dzor,"POLYGON ((45.10953 39.86578, 45.15686 39.8893,..."
5,https://simplemaps.com,AMSU,Syunik,"POLYGON ((45.70907 39.56842, 45.70892 39.5687,..."
6,https://simplemaps.com,AMAR,Ararat,"POLYGON ((45.02979 40.16334, 45.02483 40.1346,..."
7,https://simplemaps.com,AMAG,Aragatsotn,"POLYGON ((44.16581 40.74054, 44.17883 40.74273..."
8,https://simplemaps.com,AMAV,Armavir,"POLYGON ((44.44047 40.15336, 44.43825 40.14489..."
9,https://simplemaps.com,AMKT,Kotayk,"POLYGON ((44.77249 40.66517, 44.79657 40.65008..."


In [11]:
frames_dir = 'drought_data_output/frames_agri_stress'
os.makedirs(frames_dir, exist_ok=True)
filenames = []

# ensure years sorted
years = list(pivot_df.columns)
years = sorted(years)

for year in years:
    df_year = pivot_reset[[idx_name, year]].rename(columns={year: 'value'})
    if 'name' in gdf.columns:
        merged = gdf.merge(df_year, left_on='name', right_on=idx_name, how='left')
    else:
        left_key = [c for c in gdf.columns if c.lower() != 'geometry'][0]
        merged = gdf.merge(df_year, left_on=left_key, right_on=idx_name, how='left')

    fig, ax = plt.subplots(figsize=(8, 8))
    merged.plot(column='value', ax=ax, cmap=cmap, vmin=vmin, vmax=vmax,
                edgecolor='black', linewidth=0.5, legend=True,
                missing_kwds={'color': 'lightgrey', 'label': 'no data'})
    ax.set_title(f'Annual agricultural stress — {year}')
    ax.axis('off')

    fname = os.path.join(frames_dir, f'frame_{year}.png')
    fig.savefig(fname, bbox_inches='tight', dpi=150)
    plt.close(fig)
    filenames.append(fname)

# build gif
images = [imageio.imread(f) for f in filenames]
imageio.mimsave('drought_data_output/agri_stress.gif', images, duration=0.8)

# cleanup frame files if you want (uncomment to remove)
# for f in filenames:
#     os.remove(f)

print(f'GIF saved to agri_stress.gif ({len(filenames)} frames)')


C:\Users\jonas\AppData\Local\Temp\ipykernel_32160\3475543304.py:30: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(f) for f in filenames]


GIF saved to agri_stress.gif (41 frames)
